In [ ]:
import torch
from torchvision import transforms 
import torch.optim as optim
from torch.autograd import Variable
import torchvision

from vae_categorical import VAE

In [ ]:
input_dim = 28 * 28 # dimensions of each image in MNIST dataset

batch_size = 100
hidden1_size = 512
hidden2_size = 256
K = 10 #number of classes
N = 20 #number of categorical distributions
tau0 = 1 # initial temperature
epochs = 100

In [ ]:
iscuda = torch.cuda.is_available()  #check the availability of GPU

# MNIST Dataset download
transform = transforms.Compose([transforms.ToTensor()])
mnist = torchvision.datasets.MNIST('./', download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(mnist, batch_size=batch_size, shuffle=True, num_workers=2)

# instantiate VAE class object
vae = VAE(K, N, tau0, input_dim, [hidden1_size, hidden2_size], iscuda)
if iscuda:
    vae.cuda()
optimizer = optim.Adam(vae.parameters(), lr=1e-2)

In [ ]:
# training
for e in range(1,epochs):
    l = 0
    rl = 0
    kl = 0
    for i, data in enumerate(dataloader, 0):
        inputs, _ = data
        if iscuda:
            inputs = inputs.cuda()
   
        inputs = Variable(inputs.resize_(batch_size, input_dim))
        optimizer.zero_grad()
        outputs = vae(inputs)
        loss = vae.loss_fn(inputs, outputs)
        l += loss.data[0]
        rl += vae.recons_loss
        kl += vae.kl_loss
   
        loss.backward()
        optimizer.step()
   
    print "epoch: ", e, ", error: ", l/i, ", recons error: ", rl/i, ", kl divergence: ", kl/i